In [ ]:
import pandas as pd
from sklearn.datasets import load_iris 

iris = load_iris()

df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

print(df.head())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch

X = df.drop('target', axis=1).values
y = df['target'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

print("Train Features shape:", X_train_tensor.shape)
print("Train Labels shape:", y_train_tensor.shape)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class IrisNet(nn.Module):
    def __init__(self):
        super (IrisNet, self).__init__()
        self.fc1 = nn.Linear(4,10)
        self.fc2 = nn.Linear(10,3)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
    
model = IrisNet()
print(model)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05)

def calculate_accuracy(X, y):
    with torch.no_grad():
        outputs = model(X)
        _, predicted = torch.max(outputs , 1)
        accuracy = (predicted == y).sum().item() / y.size(0)
    return accuracy

train_accuracies = []
test_accuracies = []

for epoch in range(50):
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_acc = calculate_accuracy(X_train_tensor, y_train_tensor)
    test_acc = calculate_accuracy(X_test_tensor, y_test_tensor)

    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/50], Loss: {loss.item():.4f}, Train Acc: {train_acc:.2f}, Test Acc: {test_acc:.2f}") 

In [ ]:
import matplotlib.pyplot as plt

epochs = list(range(1, 51))

plt.figure(figsize=(10, 5))
plt.plot(epochs, train_accuracies, label='Train Accuracy', marker='o')
plt.plot(epochs, test_accuracies, label='Test Accuracy', marker='x')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training vs Testing Accuracy")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Mapping from label to flower name
class_names = ['Setosa', 'Versicolor', 'Virginica']

# Function to take user input and predict
def predict_flower():
    print("Enter flower features:")
    sl = float(input("Sepal Length (cm): "))
    sw = float(input("Sepal Width (cm): "))
    pl = float(input("Petal Length (cm): "))
    pw = float(input("Petal Width (cm): "))

    # Make input as a 2D array and scale using the same scaler used for training
    input_data = [[sl, sw, pl, pw]]
    input_scaled = scaler.transform(input_data)
    input_tensor = torch.tensor(input_scaled, dtype=torch.float32)

    # Model prediction
    with torch.no_grad():
        output = model(input_tensor)
        _, predicted = torch.max(output, 1)
        predicted_class = predicted.item()

    print(f"\n Predicted Flower: {class_names[predicted_class]}\n")

# Run it
predict_flower()